In [1]:
# Import Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pandas_profiling as pp
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import cohen_kappa_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot
import zipfile

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Conv1D
from keras.layers import BatchNormalization
from keras.layers import MaxPool1D
from keras.layers import Flatten
#from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

Using TensorFlow backend.


In [2]:
import os
colab = os.environ.get('COLAB_GPU', '10')
if (int(colab) == 0):
    from google.colab import drive
    drive.mount('/content/drive')
    #Path="drive/My Drive/Colab Notebooks/StockAnalysis"
    Path="" # ???
    DataPath="/content"
    RootPath="/root"    
else:
    #Path="/c/DataScience/Repo/Imbalanced_data"
    Path="/DataScience/Repo/Imbalanced_data"
    DataPath=Path
    RootPath="/Users/iowahawk89"

In [3]:
# Load the Data Set
#df = pd.read_csv('creditcard.csv')
df = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')

In [4]:
X = df.loc[:, df.columns != 'Class']
y = df.loc[:, df.columns == 'Class']
X_train, X_test1, y_train, y_test1 = train_test_split(X,y, test_size = 0.4, random_state = 42)
X_test, X_val, y_test, y_val = train_test_split(X_test1,y_test1, test_size = 0.4, random_state = 42)
class_names=[0,1] 

In [5]:
# downsampling
yes = len(y_train[y_train['Class'] ==1]) 
yes_ind = y_train[y_train['Class'] == 1].index
no_ind = y_train[y_train['Class'] == 0].index
new_no_ind = np.random.choice(no_ind, yes, replace = False)
undersample_ind = np.concatenate([new_no_ind, yes_ind])
X_train = X_train.loc[undersample_ind]
y_train = y_train.loc[undersample_ind]

In [6]:
def display_metrics(model_name, train_features, test_features, train_label, test_label, pred):
    try:
        print(model_name.score(test_features, test_label)) 
        print("Accuracy score (training): {0:.3f}".format(model_name.score(train_features, train_label))) 
        print("Accuracy score (validation): {0:.3f}".format(model_name.score(test_features, test_label))) 
    except Exception as e:
        print("error")  
    try:
        print(pd.Series(model_name.feature_importances_, index=train_features.columns[:]).nlargest(10).plot(kind='barh')) 
    except Exception as e:
        print("error")  

    print("Confusion Matrix:")
    tn, fp, fn, tp = confusion_matrix(test_label, pred).ravel()
    total = tn+ fp+ fn+ tp 
    print("false positive pct:",(fp/total)*100) 
    print("tn", " fp", " fn", " tp") 
    print(tn, fp, fn, tp) 
    print(confusion_matrix(test_label, pred)) 
    print("Classification Report") 
    print(classification_report(test_label, pred))
    print("Specificity =", tn/(tn+fp))
    print("Sensitivity =", tp/(tp+fn))
    return tn, fp, fn, tp

In [7]:
X_train

,Time_Series,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
72290,54644.0,-0.552402,0.604515,2.444300,0.250465,-0.632696,0.121463,-0.008882,0.219440,0.351241,...,-0.099959,0.052644,0.309141,-0.058075,0.434920,-0.334822,0.271962,-0.043964,0.062385,9.95
212990,139068.0,-1.108267,2.027164,-1.297249,-0.248834,0.543001,-1.792010,0.544730,0.535613,-0.563645,...,-0.355535,0.275368,0.551664,-0.363162,-0.185057,0.153307,-0.152869,-0.215549,0.068467,1.00
52084,45264.0,1.255760,-0.144373,-1.008024,-0.528532,1.933700,3.338554,-0.572602,0.833685,0.009527,...,0.072650,-0.347213,-1.230347,0.124469,0.983740,0.279849,0.046523,-0.027521,0.019498,28.62
23217,32641.0,-0.677805,0.825292,2.449632,1.378949,-0.006691,0.683210,0.307176,0.357547,-0.709962,...,-0.084112,0.084616,0.246048,-0.071007,-0.017692,-0.331228,-0.273236,0.134634,0.146676,37.16
19342,30199.0,1.103256,0.211745,0.412680,1.120798,-0.220757,-0.342137,0.058428,0.142478,-0.206612,...,-0.250141,-0.390954,-1.295671,0.276683,0.105581,0.059542,-0.833215,0.010379,0.016976,19.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42674,41194.0,-7.896886,5.381020,-8.451162,7.963928,-7.862419,-2.376820,-11.949723,5.051356,-6.912076,...,0.645974,2.557944,0.926278,0.032795,0.638073,0.361887,0.444577,1.101923,0.205958,1.52
33276,37167.0,-7.923891,-5.198360,-3.000024,4.420666,2.272194,-3.394483,-5.283435,0.131619,0.658176,...,-2.169811,-0.734308,-0.599926,-4.908301,0.410170,-1.167660,0.520508,1.937421,-1.552593,12.31
182992,125612.0,1.889618,1.073099,-1.678018,4.173268,1.015516,-0.009389,-0.079706,0.064071,-0.714517,...,-0.153570,0.203728,0.733796,-0.036560,0.334306,0.147171,0.279556,0.031669,0.035883,3.22
154697,102625.0,-4.221221,2.871121,-5.888716,6.890952,-3.404894,-1.154394,-7.739928,2.851363,-2.507569,...,-0.227882,1.620591,1.567947,-0.578007,-0.059045,-1.829169,-0.072429,0.136734,-0.599848,7.59


In [9]:
nrows, ncols = X_train.shape # (602,30)
X_train_arr = X_train.to_numpy()
y_train_arr = y_train.to_numpy()
X_train_arr = X_train_arr.reshape(nrows, ncols, 1)

nrows, ncols = X_test.shape # (602,30)
X_test_arr = X_test.to_numpy()
y_test_arr = y_test.to_numpy()
X_test_arr = X_test_arr.reshape(nrows, ncols, 1)

In [10]:
input_shape = (ncols, 1)
print(input_shape)

(30, 1)


In [11]:
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})

Using a 1D Convolution, which is useful for non-image data, such as time series related data (audio, sensor)
Be sure to remember to include an activation function in the initial Conv1D layer, otherwise the model will predict all minority case of 1's (FP rate is almost 100%) !!!

In [12]:
epochs = 200 # maybe 200?
model = Sequential()
#model.add(Conv1D(32, 2, activation = 'relu', input_shape = input_shape))
#model.add(Conv1D(filters=32, kernel_size=2, input_shape = (30) ))
model.add(Conv1D(filters=32, kernel_size=10, strides=1, activation='swish', padding='valid', input_shape=input_shape ))
# TypeError: 'int' object is not iterable
model.add(BatchNormalization())
model.add(MaxPool1D(2))
model.add(Dropout(0.2))
model.add(Conv1D(64, 2, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool1D(2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))
#model.compile(optimizer='adam', lr=0.001, loss = 'binary_crossentropy', metrics=['accuracy'])
# or
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)
# Let's train the model using RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

print(model.summary())

#class_weight = {1: 0.75,
#                0: 0.25}

#history = model.fit(X_train_arr, y_train_arr, epochs=epochs, validation_data=(X_test, y_test), verbose=1)
history = model.fit(X_train_arr, y_train_arr, epochs=epochs, verbose=1, sample_weight=np.where(y_train_arr == 1,6.0,1.0).flatten())
#history = model.fit(X_train_arr, y_train_arr, epochs=epochs, verbose=1, sample_weight=class_weight)
#plot_learningCurve(history, epochs)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 21, 32)            352       
_________________________________________________________________
batch_normalization_1 (Batch (None, 21, 32)            128       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 10, 32)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 9, 64)             4160      
_________________________________________________________________
batch_normalization_2 (Batch (None, 9, 64)             256       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 4, 64)            

In [13]:
# predict probabilities for test set
yhat_probs = model.predict(X_test_arr, verbose=0)
# predict crisp classes for test set
yhat_classes = model.predict_classes(X_test_arr, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]

tn, fp, fn, tp = display_metrics(model, X_train_arr, X_test_arr, y_train_arr, y_test_arr, yhat_classes)
#visualize(y_test_arr, yhat_classes, 'CNN Simple')

error
error
Confusion Matrix:
false positive pct: 15.352654601846297
tn  fp  fn  tp
57740 10494 7 112
[[57740 10494]
 [    7   112]]
Classification Report
              precision    recall  f1-score   support

           0       1.00      0.85      0.92     68234
           1       0.01      0.94      0.02       119

    accuracy                           0.85     68353
   macro avg       0.51      0.89      0.47     68353
weighted avg       1.00      0.85      0.92     68353

Specificity = 0.8462057039012809
Sensitivity = 0.9411764705882353


In [15]:
fpr, tpr, thresholds = metrics.roc_curve(y_test_arr, yhat_classes, pos_label=1)
print('TEST | AUC Score: ' + str((metrics.auc(fpr, tpr))))

TEST | AUC Score: 0.8936910872447581
